# Advent of Code 2022

#### Preparation

## Day 1: Calorie Counting

In [20]:
with open('01/input.txt', 'r') as file:    
    lines = [line.strip() for line in file.readlines()]
    lines.append('')
    
elf, elfes = 0, []
for line in lines:
    if line == '':
        elfes.append(elf)
        elf = 0
    else:
        elf += int(line)
            
elfes.sort(reverse=True)

print(f'FIRST:  {elfes[0]}')
print(f'SECOND: {sum(elfes[:3])}')

FIRST:  70296
SECOND: 205381


## Day 2: Rock Paper Scissors

In [22]:
# Handmade permutations, manageable effort
D1 = {
    'A X': 4, # Draw
    'A Y': 8, # Win
    'A Z': 3, # Lose

    'B X': 1, # Lose
    'B Y': 5, # Draw
    'B Z': 9, # Win

    'C X': 7, # Win
    'C Y': 2, # Lose
    'C Z': 6, # Draw
}

D2 = {
    'A X': 3, # Lose
    'A Y': 4, # Draw
    'A Z': 8, # Win

    'B X': 1, # Lose
    'B Y': 5, # Draw
    'B Z': 9, # Win

    'C X': 2, # Lose
    'C Y': 6, # Draw
    'C Z': 7, # Win
}

with open('02/input.txt', 'r') as file:
    ret1, ret2 = 0, 0
    for line in file.readlines():
        line_stripped = line.strip()
        ret1 += D1[line_stripped]
        ret2 += D2[line_stripped]
        
print(f'FIRST:  {ret1}')
print(f'SECOND: {ret2}')

FIRST:  13484
SECOND: 13433


## Day 3: Rucksack Reorganization

In [30]:
with open('03/input.txt', 'r') as file:
    lines = [line.strip() for line in file.readlines()]
    
# Primera estrella
ret1 = 0
for line in lines:
    left_set = set(line[:len(line) // 2])
    right_set = set(line[len(line) // 2:])
    # Get intersection
    ch = (left_set & right_set).pop()
    ret1 += ord(ch) - 96 if ch.islower() else ord(ch) - 38
    
# Segunda estrella
i, ret2 = 0, 0
while i < len(lines):
    set_a, set_b, set_c = [set(s) for s in lines[i:i+3]]
    # Only one element in intersection, therefore .pop() will suffice
    ch = (set_a & set_b & set_c).pop()
    ret2 += ord(ch) - 96 if ch.islower() else ord(ch) - 38
    i += 3

print(f'FIRST:  {ret1}')
print(f'SECOND: {ret2}')

FIRST:  7746
SECOND: 2604


 ## Day 4: Camp Cleanup

In [32]:
with open('04/input.txt', 'r') as file:
    ret1, ret2 = 0, 0
    for line in file.readlines():
        left, right = line.strip().split(',')
        a, b = left.split('-')
        c, d = right.split('-')
        a, b, c, d = [int(i) for i in [a, b, c, d]]
        # First
        if (c >= a and d <= b) or (a >= c and b <= d):
            ret1 += 1
        # Second
        if (a <= c <= b) or (c <= a <= d):
            ret2 += 1

print(f'FIRST:  {ret1}')
print(f'SECOND: {ret2}')

FIRST:  424
SECOND: 804


## Day 5: Supply Stacks

In [5]:
from copy import deepcopy

with open('05/input.txt', 'r') as file:
    st, instructions = file.read().split('\n\n')
    st, instructions = st.split('\n'), instructions.split('\n')
    
# Initialize empty stacks with number of ints in the last line of stack declarations (st)
stacksA = [[] for _ in range(len(st[-1].split()))]

# Traverse rows and fill stacks
for j, c in enumerate(st[-1]):
    if c.isnumeric():
        for i in range(len(st)-2, -1, -1):
            if st[i][j].isalpha():
                stacksA[int(c)-1].append(st[i][j])

stacksB = deepcopy(stacksA)

for line in instructions[:-1]:
    _, amount, _, frm, _, to = line.split()
    amount, frm, to = [int(i) for i in [amount, frm, to]]
    
    # First
    for _ in range(amount):
        stacksA[to-1].append(stacksA[frm-1].pop())
    # Second
    cut = stacksB[frm-1][-amount:]
    del stacksB[frm-1][-amount:]
    for ch in cut:
        stacksB[to-1].append(ch)

FIRST  = ''.join([stack[-1] for stack in stacksA])
SECOND = ''.join([stack[-1] for stack in stacksB])

print(f'FIRST:  {FIRST}')
print(f'SECOND: {SECOND}')

FIRST:  MQTPGLLDN
SECOND: LVZPSTTCZ


## Day 6: Tuning Trouble

In [12]:
with open('06/input.txt', 'r') as file:
    line = file.readline()

FIRST = 4
SECOND = 14

for i in range(len(line) - FIRST):
    if len(set(line[i:i+FIRST])) == FIRST:
        print(f'FIRST:  {i + FIRST}')
        break

for i in range(len(line) - SECOND):
    if len(set(line[i:i+SECOND])) == SECOND:
        print(f'SECOND: {i + SECOND}')
        break

FIRST:  1623
SECOND: 3774


## Day 7: No Space Left On Device

In [6]:
class Node():
    def __init__(self, name):
        self.name = name
        self.parent = None
        self.files = {}
        self.children = []
        self.node_sum = 0
        self.first_sum = 0
        self.dirs = []

    def cd_dir(self, name):
        if name == '..':
            return self.parent
        for child in self.children:
            if name == child.name:
                return child
        return None

    def add_dir(self, name):
        new_child = Node(name)
        new_child.parent = self
        self.children.append(new_child)

    def add_file(self, name, size):
        self.files[name] = int(size)
        while self is not None:
            self.node_sum += int(size)
            self = self.parent

    def traverse(self, node):
        for n in node.children:
            if n:
                self.traverse(n)
                self.dirs.append(n.node_sum)
                if n.node_sum <= 100000:
                    self.first_sum += n.node_sum

    def printall(self):
        self.traverse(self)
        self.dirs.append(self.node_sum)
        self.dirs.sort()
        total_disk_space  = 70000000
        target_free_space = 30000000
        print(f'\nFIRST:  {self.first_sum}')
        free_space = total_disk_space - self.dirs[-1]
        for dir in self.dirs:
            if (free_space + dir) >= target_free_space:
                print(f'SECOND: {dir}')
                break


with open('07/input.txt', 'r') as file:
    lines = [line.strip() for line in file.readlines()]

_, _, path = lines[0].split()
root = Node(path)
cwd = root

for line in lines[1:]:
    l = line.split()
    if len(l) == 3:
        cd_dir = cwd.cd_dir(l[-1])
        if cd_dir is not None:
            cwd = cd_dir
        else:
            raise Exception('cd_dir() error!')
    else:
        if l[-1] == 'ls':
            continue
        if l[0] == 'dir':
            cwd.add_dir(l[-1])
        else:
            cwd.add_file(l[-1], l[0])

root.printall()


FIRST:  1642503
SECOND: 6999588


## Day 8: Treetop Tree House

In [1]:
with open('08/input.txt', 'r') as file:
    lines = tuple(tuple(int(j) for j in line.strip()) for line in file.readlines())

neighbors = (
    (-1, 0), # NORTH
    (0, 1),  # EAST
    (1, 0),  # SOUTH
    (0, -1), # WEST
)

N = len(lines[0])

def in_bounds(i: int, j: int) -> bool:
    return (0 <= i < N) and (0 <= j < N)

def radiate(i: int, j: int, di: int, dj: int, value: int) -> tuple[bool, int]:
    # Increase/decrease i/j by the delta to radiate in all 4 cardinal directions
    i, j = i + di, j + dj
    count = 0
    has_foresight = True
    while in_bounds(i, j):
        count += 1
        if lines[i][j] >= value:
            has_foresight =  False
            break
        i, j = i + di, j + dj
    return (has_foresight, count)

count1, count2 = 0, 0

# Skip outer rows
for i in range(1, len(lines) - 1):
    # Skip outer columns
    for j in range(1, len(lines[i]) - 1):
        flag = False
        interim = 1
        for n in neighbors:
            res =  radiate(i, j, n[0], n[1], lines[i][j])
            interim *= res[1]
            if res[0] and not flag:
                count1 +=1
                flag = True
        count2 = max(interim, count2)

print(f'FIRST:  {count1 + 2 * N + 2 * N - 4}')
print(f'SECOND: {count2}')

FIRST:  1708
SECOND: 504000


## Day 9: Rope Bridge

In [3]:
from dataclasses import dataclass
import sys
import math


path = []
tail = tuple((0, 0))
intersections = set((tail))

@dataclass
class Coordinates:
    i: int
    j: int

current = Coordinates(0, 0)

def extend_path(direction: str, n: int) -> None:
    match direction:
        case 'U': di, dj = -1, 0
        case 'R': di, dj = 0, 1
        case 'D': di, dj = 1, 0
        case  _ : di, dj = 0, -1

    for _ in range(n):
        current.i += di
        current.j += dj
        path.append(tuple((current.i, current.j)))


def eucl(tail: tuple, head: tuple) -> int:
    return int(math.sqrt((tail[0] - head[0])**2 + (tail[1] - head[1])**2))


with open('09/input.txt', 'r') as file:
    for line in file.readlines():
        direction, n = line.split()
        extend_path(direction, int(n))


S = set()
count = 0
for i in range(len(path) - 1):
    head = path[i + 1]
    count += 1
    if eucl(tail, head) > 1:
        tail = path[i]
        # Own hash function
        intersections.add(tail[0] * 0xAFFE + tail[1])


print(f'FIRST:  {len(intersections)}')

FIRST:  6642
